# JSON Messages

This feels more cumberson to write in json.

In [1]:
# reload library
%load_ext autoreload
%autoreload 2

In [2]:
from gecko_messages.utils import *
from gecko_messages.c_generator import *
from gecko_messages.py_generator import *
from gecko_messages.builtins import *
from gecko_messages.files import read_folder

In [3]:
import json
from pprint import pprint
from collections import namedtuple
from types import SimpleNamespace

In [4]:
from pygments import highlight
from pygments.lexers import PythonLexer, CppLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML, display

In [5]:
def print_c(content):
    display(HTML(highlight(content, CppLexer(), HtmlFormatter())))

def print_python(content):
    display(HTML(highlight(content, PythonLexer(), HtmlFormatter())))

In [16]:
# Type = namedtuple("Type","size fmt")
# vec_t = VarInfo("vec_t",    "vec_t",    12, "3f", True)
vec_t = {"type": "vec_t", "default": [0,0,0]}
Float = {"type": "float"}
Uint32 = {"type": "uint32"}

In [10]:
def var_fix_json(data):
    """
    format variables in message properly from the TOML template
    float-x => float x
    float_3_x => float[3] x
    """
    vars = []
    key_remove = []
    # m = re.compile(r'([a-zA-Z0-9_]+)')
    for k,v in data["message"].items():
        # print(k, v)
        try:
            # there are some keys that are not variable types
            # and we are just protecting them here
            if k == "id":
                continue
            if k == "name":
                continue

            key_remove.append(k)
            var = k
            type = v["type"]
            array_size = v.get("len", 0)
            default = v.get("default", 0)
            var = Var(type,var,default,int(array_size))
            vars.append(var)
        except ValueError:
            continue

    # Add in new namedtuple Var array
    data["message"]["vars"] = vars

    # remove old keys that were replaced above with namedtuple code
    for k in key_remove:
        data["message"].pop(k)

    return data

In [17]:
global_json = {
    "global": {
        "namespace": "msgs",
        "license": "hello",
        "serialize": {
            "yivo": False,
            "mavlink": False
        },
        "ids": {
            "imu_t": 10
        }
    }
}

msgjson = {
    "message": {
        "accels": vec_t,
        "gyros": vec_t,
        "mags": vec_t,
        "pressure": {
            "type": "float",
            "len": 6
        },
        "temperature": Float,
        "time": Uint32,
        "name": "imu_t"
    }
}

msgjson = msgjson | global_json

pprint(msgjson)
print("------------------------")
msg = var_fix_json(msgjson)
pprint(msg)

{'global': {'ids': {'imu_t': 10},
            'license': 'hello',
            'namespace': 'msgs',
            'serialize': {'mavlink': False, 'yivo': False}},
 'message': {'accels': {'default': [0, 0, 0], 'type': 'vec_t'},
             'gyros': {'default': [0, 0, 0], 'type': 'vec_t'},
             'mags': {'default': [0, 0, 0], 'type': 'vec_t'},
             'name': 'imu_t',
             'pressure': {'len': 6, 'type': 'float'},
             'temperature': {'type': 'float'},
             'time': {'type': 'uint32'}}}
------------------------
{'global': {'ids': {'imu_t': 10},
            'license': 'hello',
            'namespace': 'msgs',
            'serialize': {'mavlink': False, 'yivo': False}},
 'message': {'name': 'imu_t',
             'vars': [Var(type='vec_t', var='accels', value=[0, 0, 0], array_size=0),
                      Var(type='vec_t', var='gyros', value=[0, 0, 0], array_size=0),
                      Var(type='vec_t', var='mags', value=[0, 0, 0], array_size=0),
        

## C

In [18]:
content = create_cpp(msg)
# print(content)
print_c(content)

## Python

In [19]:
content = create_python(msg)
# print(content)
print_python(content)